<center><h1>The astoundingly unequalty in the time of COVID-19</h1></center>
<br>

<br>

<h1 style='color:black'><center>Introduction</center></h1> 

Over two years into the COVID-19 pandemic, we have a growing arsenal of tools to fight the virus. But we are still lacking a truly global and equitable response to help find a way out of the pandemic.

Safe and effective vaccines have been available for over a year, so why aren’t vaccination rates increasing everywhere in the world? It comes down to three factors: supply, distribution, and last-mile logistics of getting shots into arms.

Rich countries bought the majority of the initial supply of vaccines, leaving low- and lower middle-income countries out of the market for over a year. The global supply of vaccines has only just begun to meaningfully increase — one year after they were made available in high-income countries.
<br>

<h1 style='color:black'><center>Data collection</center></h1> 

The data is collected by [Our World in Data](https://ourworldindata.org) by browsing public information from official sources, including:

* **Confirmed cases**    
* **Confirmed deaths** - Due to varying protocols and challenges in the attribution of the cause of death, the number of confirmed deaths may not accurately represent the true number of deaths caused by COVID-19. 
* **Excess mortality** 
* **Hospital & ICU** 
* **Policy responses**
* **Reproduction rate** 
* **Tests & positivity** - On 23 June 2022, [Our World in Data](https://ourworldindata.org) stopped adding new datapoints to COVID-19 testing dataset  
* **Vaccinations** - 
* **Others** 

And from [the World Bank](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups)'s income classification, including:

* **Classification by income** 

<a id="0"></a>



In [1]:
import pandas as pd
import plotly.express as px
import plotly as ply
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sci
import plotly.io as pio
import datetime
import numpy as np
from sklearn import linear_model
from IPython.display import Markdown, display, HTML
from tabulate import tabulate
import math
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

pio.renderers.default = "jupyterlab"

millnames = ['',' Thousand',' Million',' Billion',' Trillion']
def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    return '{:.2f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

def printResult(df):
        print()
        df.index.name = '&nbsp;'
        string = tabulate(df, headers='keys', tablefmt='github', showindex="always")
        display(Markdown(string))
        print()
        
#The complete Our World in Data COVID-19 dataset 
#Source: https://ourworldindata.org/covid-vaccinations
df_owid_covid_data = pd.read_csv('../input/owidcoviddata/owid-covid-data.csv')
df_owid_covid_data = df_owid_covid_data[['iso_code','continent','location','date','total_vaccinations','people_vaccinated','people_fully_vaccinated','total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']]

# Classification by income
#Source: https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups
df_world_income = pd.read_csv('../input/worldincome/WorldIncome.csv')
df_world_income = df_world_income.drop(['Region'], axis=1)

#Merge df_owid_covid_data and Classification by income
df = pd.merge(df_owid_covid_data, df_world_income, on='iso_code', how="outer")

print("DataFrame csv 1:")
print("Num elementi violence_data csv: {}".format(len(df_owid_covid_data)))
printResult(df_owid_covid_data.head(3))
print()
print("DataFrame csv 2:")
print("Num elementi countries csv: {}".format(len(df_world_income)))
printResult(df_world_income.head(3))
print()
print("DataFrame finale:")
print("Num elementi dopo merge: {}".format(len(df)))
printResult(df.head(3))   
    


date_total_vaccinations = df[(df['iso_code'] == 'OWID_WRL') & (df['total_vaccinations'] == df['total_vaccinations'].max())]
low_income = df[(df['iso_code'] == 'OWID_LIC') & (pd.notna(df['total_vaccinations']))].max()
lower_middle_income = df[(df['iso_code'] == 'OWID_LMC') & (pd.notna(df['total_vaccinations']))].max()
upper_middle_income = df[(df['iso_code'] == 'OWID_UMC') & (pd.notna(df['total_vaccinations']))].max()
high_income = df[(df['iso_code'] == 'OWID_HIC') & (pd.notna(df['total_vaccinations']))].max()

africa = df[(df['iso_code'] == 'OWID_AFR') & (pd.notna(df['total_vaccinations']))].max()


display(Markdown("<h3>As of {}, <mark>{}</mark> vaccine doses have been administered in the world:</h3>".format(pd.to_datetime(date_total_vaccinations['date']).dt.strftime('%d %B %Y').values[0],millify(date_total_vaccinations['total_vaccinations'] ))))
display(Markdown("* **<mark>{}</mark>** doses in low-income countries".format(millify(low_income['total_vaccinations']))))
display(Markdown("* **<mark>{}</mark>** doses in lower middle-income countries".format(millify(lower_middle_income['total_vaccinations']))))
display(Markdown("* **<mark>{}</mark>** doses in upper middle-income countries".format(millify(upper_middle_income['total_vaccinations']))))
display(Markdown("* **<mark>{}</mark>** doses in high-income countries".format(millify(high_income['total_vaccinations']))))

display(Markdown("African countries have administered <mark>{}</mark> doses. That means <mark>{}%</mark> of the population of the continent is fully vaccinated.".format(millify(africa['total_vaccinations']),africa['people_fully_vaccinated_per_hundred'])))

display(Markdown("To date, <mark>{}%</mark> of the world’s population has been fully vaccinated. But only <mark>{}%</mark> of people in low-income countries have been fully vaccinated. Lower middle income countries have fully vaccinated <mark>{}%</mark> of their people. That’s a huge difference compared with <mark>{}%</mark> in high income countries, and <mark>{}%</mark> in upper middle-income countries.".format(date_total_vaccinations['people_fully_vaccinated_per_hundred'].values[0],low_income['people_fully_vaccinated_per_hundred'],lower_middle_income['people_fully_vaccinated_per_hundred'],high_income['people_fully_vaccinated_per_hundred'],upper_middle_income['people_fully_vaccinated_per_hundred'])))

DataFrame csv 1:
Num elementi violence_data csv: 201499



|   &nbsp; | iso_code   | continent   | location    | date       |   total_vaccinations |   people_vaccinated |   people_fully_vaccinated |   total_vaccinations_per_hundred |   people_vaccinated_per_hundred |   people_fully_vaccinated_per_hundred |
|----------|------------|-------------|-------------|------------|----------------------|---------------------|---------------------------|----------------------------------|---------------------------------|---------------------------------------|
|        0 | AFG        | Asia        | Afghanistan | 2020-02-24 |                  nan |                 nan |                       nan |                              nan |                             nan |                                   nan |
|        1 | AFG        | Asia        | Afghanistan | 2020-02-25 |                  nan |                 nan |                       nan |                              nan |                             nan |                                   nan |
|        2 | AFG        | Asia        | Afghanistan | 2020-02-26 |                  nan |                 nan |                       nan |                              nan |                             nan |                                   nan |



DataFrame csv 2:
Num elementi countries csv: 218



|   &nbsp; | iso_code   | Country     | Income_group        |
|----------|------------|-------------|---------------------|
|        0 | ABW        | Aruba       | High income         |
|        1 | AFG        | Afghanistan | Low income          |
|        2 | AGO        | Angola      | Lower middle income |



DataFrame finale:
Num elementi dopo merge: 201503



...

* **<mark>4.25 Billion</mark>** doses in lower middle-income countries

* **<mark>5.25 Billion</mark>** doses in upper middle-income countries

* **<mark>2.52 Billion</mark>** doses in high-income countries

African countries have administered <mark>581.42 Million</mark> doses. That means <mark>20.29%</mark> of the population of the continent is fully vaccinated.

To date, <mark>61.26%</mark> of the world’s population has been fully vaccinated. But only <mark>15.79%</mark> of people in low-income countries have been fully vaccinated. Lower middle income countries have fully vaccinated <mark>55.15%</mark> of their people. That’s a huge difference compared with <mark>73.46%</mark> in high income countries, and <mark>78.67%</mark> in upper middle-income countries.

In [2]:
df_world = df.groupby(['location', 'iso_code'], as_index=False)['total_vaccinations_per_hundred'].last()

df_world = df_world[~df_world["iso_code"].str.contains('OWID')]

fig = px.choropleth(df_world, locations='iso_code', color='total_vaccinations_per_hundred', hover_name='location', 
                    color_continuous_scale=px.colors.sequential.Plasma, projection="natural earth")
fig.update_layout(
    title={
    'text': "Media valori DHS per paese",
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
fig.show()

In [3]:
dates = pd.date_range('2021-01-01', date_total_vaccinations['date'].values[0], freq='3MS').strftime('%Y-%m-%d')

results = pd.DataFrame()
for date in dates:
    results = pd.concat([results, df[(df['date'] == date) & (df['iso_code'].isin(['OWID_LIC','OWID_LMC','OWID_UMC','OWID_HIC']) ) ]], axis=0).reset_index(drop=True)

fig = px.line(results, x="date", y="people_fully_vaccinated_per_hundred", color='location')
fig.show()

In [4]:

df_LI = df.groupby(['location', 'iso_code','Income_group'], as_index=False)['people_fully_vaccinated_per_hundred'].last()
df_LI = df_LI[df_LI['Income_group']== "Low income"]

fig = px.bar(df_LI, y='people_fully_vaccinated_per_hundred', x='location', color = 'location', height = 800)
fig.update_layout(showlegend=False,
    title={
    'text': "Inizio sondaggi DHS per ogni paese",
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
fig.show()